In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt

/home/ic-lab/miniconda3/envs/torch_v1.9_gpu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
task_datasets = []

# Task 1
task1_data = np.load('./cwru1_X_train.npy')
task1_labels = np.load('./cwru2_y_train.npy')

# Task 2
task2_data = np.load('./cwru2_X_train.npy')
task2_labels = np.load('./cwru2_y_train.npy')

# Task 3
task3_data = np.load('./cwru3_X_train.npy')
task3_labels = np.load('./cwru3_y_train.npy')

# Task 4
task4_data = np.load('./cwru4_X_train.npy')
task4_labels = np.load('./cwru4_y_train.npy')

In [3]:
# Convert the data to PyTorch tensors
# Task 1
task1_data = torch.from_numpy(task1_data)
task1_labels = torch.from_numpy(task1_labels)

# Task 2
task2_data = torch.from_numpy(task2_data)
task2_labels = torch.from_numpy(task2_labels)

# Task 3
task3_data = torch.from_numpy(task3_data)
task3_labels = torch.from_numpy(task3_labels)

# Task 4
task4_data = torch.from_numpy(task4_data).double()
task4_labels = torch.from_numpy(task4_labels).double()

In [4]:
# data transpose
task1_data = task1_data.transpose(1, 2)
task2_data = task2_data.transpose(1, 2)
task3_data = task3_data.transpose(1, 2)
task4_data = task4_data.transpose(1, 2)

In [5]:
# dataset for task 1~4
dataset1 = torch.utils.data.TensorDataset(task1_data, task1_labels)
dataset2 = torch.utils.data.TensorDataset(task2_data, task2_labels)
dataset3 = torch.utils.data.TensorDataset(task3_data, task3_labels)
dataset4 = torch.utils.data.TensorDataset(task4_data, task4_labels)

In [6]:
# 划分训练集和测试集
train_size = int(0.8 * len(dataset1))
test_size = len(dataset1) - train_size

train_dataset1, test_dataset1 = random_split(dataset1, [train_size, test_size])
train_dataset2, test_dataset2 = random_split(dataset2, [train_size, test_size])
train_dataset3, test_dataset3 = random_split(dataset3, [train_size, test_size])
train_dataset4, test_dataset4 = random_split(dataset4, [train_size, test_size])

In [7]:
task_datasets.append((train_dataset1, test_dataset1))
task_datasets.append((train_dataset2, test_dataset2))
task_datasets.append((train_dataset3, test_dataset3))
task_datasets.append((train_dataset4, test_dataset4))

In [8]:
# 定义WDCNN模型
class WDCNN(nn.Module):
    def __init__(self, num_classes):
        super(WDCNN, self).__init__()
        self.conv1 = nn.Conv1d(2, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 256, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x

In [9]:
# 定义训练函数
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs = inputs.to(device, dtype=torch.double)
        labels = labels.to(device, dtype=torch.long)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

# 定义测试函数
def test(model, dataloader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device, dtype=torch.double)
            labels = labels.to(device, dtype=torch.long)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    epoch_loss = running_loss / len(dataloader.dataset)
    accuracy = correct / total
    return epoch_loss, accuracy

# 初始化参数
num_classes = 10
num_epochs = 5
lr = 0.001
batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
# 构建和加载第一个任务的训练和测试数据集
# task1_train_dataset = ...
# task1_test_dataset = ...
task1_train_dataloader = torch.utils.data.DataLoader(train_dataset1, batch_size=batch_size, shuffle=True)
task1_test_dataloader = torch.utils.data.DataLoader(test_dataset1, batch_size=batch_size, shuffle=False)

# 创建WDCNN模型
model = WDCNN(num_classes).to(device)
model.double()

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [11]:
# 训练第一个任务
for epoch in range(num_epochs):
    # 训练
    train_loss = train(model, task1_train_dataloader, criterion, optimizer, device)
    # 测试
    test_loss, test_accuracy = test(model, task1_test_dataloader, criterion, device)
    # 打印训练和测试结果
    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 1/5: Train Loss: 0.4883, Test Loss: 0.0893, Test Accuracy: 0.9707
Epoch 2/5: Train Loss: 0.0563, Test Loss: 0.0297, Test Accuracy: 0.9921
Epoch 3/5: Train Loss: 0.0454, Test Loss: 0.0258, Test Accuracy: 0.9904
Epoch 4/5: Train Loss: 0.0104, Test Loss: 0.0363, Test Accuracy: 0.9875
Epoch 5/5: Train Loss: 0.0055, Test Loss: 0.0714, Test Accuracy: 0.9786


In [12]:
# 其余任务测试
for task in range(1, 4):
    task_test_dataloader = torch.utils.data.DataLoader(task_datasets[task][1], batch_size=batch_size, shuffle=False)
    
    test_loss, test_accuracy = test(model, task_test_dataloader, criterion, device)
    # 打印训练和测试结果
    print(f"Task {task} - Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Task 1 - Epoch 5/5: Train Loss: 0.0055, Test Loss: 4.6439, Test Accuracy: 0.5918
Task 2 - Epoch 5/5: Train Loss: 0.0055, Test Loss: 6.0222, Test Accuracy: 0.5725
Task 3 - Epoch 5/5: Train Loss: 0.0055, Test Loss: 4.4044, Test Accuracy: 0.5950


In [13]:
# 冻结卷积层参数
for param in model.conv1.parameters():
    param.requires_grad = False
for param in model.conv2.parameters():
    param.requires_grad = False

In [14]:
# # 创建新的分类器并保存与任务相关的模型
# task1_classifier = nn.Sequential(
#     nn.Linear(64 * 256, 128),
#     nn.ReLU(),
#     nn.Linear(128, num_classes)
# )
# # task1_classifier.load_state_dict(model.fc.state_dict())
# torch.save(task1_classifier.state_dict(), "./parameters/task1_classifier.pt")

# 创建新的分类器并保存与任务相关的模型
task1_classifier = nn.Linear(128, num_classes).to(device)
task1_classifier.load_state_dict(model.fc2.state_dict())
torch.save(task1_classifier.state_dict(), "./parameters/task1_classifier.pt")

In [18]:
import random

In [33]:
# 加载并适应其余任务
for task in range(1, 4):
    # 构建和加载第task任务的训练和测试数据集
    # task_train_dataset = ...
    # task_test_dataset = ...
    # print(f'train_dataset{task}')
    sampled_train_dataset = random.sample(list(task_datasets[task][0]), 50)
    # print(task_datasets[task][0][:50])
    task_train_dataloader = torch.utils.data.DataLoader(sampled_train_dataset, batch_size=batch_size, shuffle=True)
    task_test_dataloader = torch.utils.data.DataLoader(task_datasets[task][1], batch_size=batch_size, shuffle=False)
    # print(task_train_dataloader)
    
    # 加载任务相关的分类器
    classifier = nn.Linear(128, num_classes).to(device)
    classifier.load_state_dict(torch.load(f"./parameters/task1_classifier.pt"))

    # 替换模型的分类器
    model.fc2 = classifier
    model.double()

    # 重新定义优化器
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # 训练
    for epoch in range(num_epochs):
        # 训练
        train_loss = train(model, task_train_dataloader, criterion, optimizer, device)
        # 测试
        test_loss, test_accuracy = test(model, task_test_dataloader, criterion, device)
        # 打印训练和测试结果
        print(f"Task {task+1} - Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

    # 保存与任务相关的模型
    # torch.save(model.state_dict(), f"task{task}_model.pt")

Task 2 - Epoch 1/5: Train Loss: 1.1132, Test Loss: 0.4157, Test Accuracy: 0.8714
Task 2 - Epoch 2/5: Train Loss: 0.2882, Test Loss: 0.2696, Test Accuracy: 0.9068
Task 2 - Epoch 3/5: Train Loss: 0.1432, Test Loss: 0.2321, Test Accuracy: 0.9268
Task 2 - Epoch 4/5: Train Loss: 0.1358, Test Loss: 0.2123, Test Accuracy: 0.9432
Task 2 - Epoch 5/5: Train Loss: 0.0955, Test Loss: 0.1875, Test Accuracy: 0.9493
Task 3 - Epoch 1/5: Train Loss: 0.0819, Test Loss: 0.1408, Test Accuracy: 0.9307
Task 3 - Epoch 2/5: Train Loss: 0.0673, Test Loss: 0.0694, Test Accuracy: 0.9879
Task 3 - Epoch 3/5: Train Loss: 0.0402, Test Loss: 0.0470, Test Accuracy: 0.9875
Task 3 - Epoch 4/5: Train Loss: 0.0232, Test Loss: 0.0529, Test Accuracy: 0.9843
Task 3 - Epoch 5/5: Train Loss: 0.0191, Test Loss: 0.0608, Test Accuracy: 0.9775
Task 4 - Epoch 1/5: Train Loss: 1.0906, Test Loss: 0.9382, Test Accuracy: 0.8429
Task 4 - Epoch 2/5: Train Loss: 0.5822, Test Loss: 0.8664, Test Accuracy: 0.7639
Task 4 - Epoch 3/5: Train Lo